In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import geopy.distance
import geopandas as gpd
from shapely.geometry import Point, Polygon


In [3]:
# Load the data
listings_df = pd.read_csv('data/listings.csv')
listingsgz_df = pd.read_csv('data/listings.csv.gz')
reviews_df = pd.read_csv('data/reviews.csv')
calendargz_df = pd.read_csv('data/calendar.csv.gz')
neighbourhoods_df = pd.read_csv('data/neighbourhoods.csv')
reviewsgz_df = pd.read_csv('data/reviews.csv.gz')
neighbourhoods_df_geojson = gpd.read_file('data/neighbourhoods.geojson')

#### Un grupo de inversionistas, solicitan un estudio sobre la ciudad de Munich cercano a Theresienwiese, donde se celebra todos los años el OctokerFest. Creen que invertir en esta zona, les dara mayores ganancias a futuro que en otras zonas de Munich. El objetivo de adquirir la propiedad, es convertirlo en un Airbnb, quieren determinar segun los resultados si será para compartir o departamento entero, según cual da mayores ganancias.
##### Se les brindará la informacion de los barrios más cercanos a este punto, con mayores precios de alquiler por Airbnb junto con el tipo de alquiler (compartido o entero).

In [4]:
listings_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,97945,Rental unit in Munich · ★4.84 · 1 bedroom · 1 ...,517685,Angelika,NaN,Hadern,48.11492,11.48954,Entire home/apt,80.0,2,127,2019-10-03,0.85,2,72,0,NaN
1,114695,Rental unit in Munich · ★4.82 · 1 bedroom · 1 ...,581737,Stephan,NaN,Berg am Laim,48.12071,11.63758,Entire home/apt,105.0,2,66,2023-10-25,0.43,3,166,11,NaN
2,127383,Rental unit in Munich · ★4.83 · 1 bedroom · 1 ...,630556,Sonja,NaN,Maxvorstadt,48.15199,11.56482,Entire home/apt,120.0,3,110,2023-12-12,0.72,1,5,3,NaN
3,159634,Rental unit in Munich · ★4.63 · 1 bedroom · 1 ...,765694,Susana,NaN,Pasing-Obermenzing,48.13855,11.46586,Entire home/apt,60.0,3,37,2023-08-09,0.25,1,5,1,NaN
4,170154,Home in Munich · ★4.96 · 1 bedroom · 2 beds · ...,108297,Alex,NaN,Sendling-Westpark,48.10814,11.52733,Private room,70.0,3,546,2023-12-19,3.66,1,169,25,NaN


In [12]:
listings_df.isnull().sum()[listings_df.isnull().sum() > 0].sort_values(ascending=False)

neighbourhood_group    6555
license                6535
price                  1447
last_review            1314
reviews_per_month      1314
host_name                 1
dtype: int64

In [6]:
listingsgz_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,14726865,https://www.airbnb.com/rooms/14726865,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 1 bed · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/7d11d1db-62ce...,91678611,...,NaN,NaN,NaN,NaN,f,1,0,0,1,NaN
1,14976237,https://www.airbnb.com/rooms/14976237,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 3 beds · 1...,NaN,NaN,https://a0.muscache.com/pictures/9f7cdeb8-3b33...,12448910,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
2,15794305,https://www.airbnb.com/rooms/15794305,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 1 bed · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/c458bf85-45bc...,44987181,...,5.0,5.0,5.0,NaN,f,1,1,0,0,0.02
3,40839352,https://www.airbnb.com/rooms/40839352,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 1 bed · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,317808258,...,5.0,5.0,5.0,NaN,f,1,1,0,0,0.05
4,45776129,https://www.airbnb.com/rooms/45776129,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 2 beds · 1...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,87534103,...,NaN,NaN,NaN,NaN,t,3,2,1,0,NaN


In [11]:
listingsgz_df.isnull().sum()[listingsgz_df.isnull().sum() > 0].sort_values(ascending=False)

description                     6555
bathrooms                       6555
calendar_updated                6555
bedrooms                        6555
neighbourhood_group_cleansed    6555
license                         6535
host_neighbourhood              4567
host_about                      3988
neighborhood_overview           3908
neighbourhood                   3908
host_response_rate              2310
host_response_time              2310
price                           1447
has_availability                1447
host_location                   1334
review_scores_checkin           1314
review_scores_accuracy          1314
review_scores_rating            1314
last_review                     1314
review_scores_communication     1314
review_scores_location          1314
review_scores_value             1314
first_review                    1314
review_scores_cleanliness       1314
reviews_per_month               1314
host_acceptance_rate            1177
beds                             107
h

In [19]:
listingsgz_df_lesscol = listingsgz_df.drop(['license', 'host_about', 'host_response_rate', 'host_response_time', "host_location","host_thumbnail_url","host_picture_url", "host_listings_count", "picture_url","description"], axis=1)

In [35]:
listingsgz_df_lesscol.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'neighborhood_overview', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of

In [46]:
listingsgz_df_lesscol["room_type"]

0           Shared room
1       Entire home/apt
2       Entire home/apt
3       Entire home/apt
4       Entire home/apt
             ...       
6550    Entire home/apt
6551    Entire home/apt
6552    Entire home/apt
6553       Private room
6554    Entire home/apt
Name: room_type, Length: 6555, dtype: object

In [33]:
listingsgz_df_lesscol["price"].replace({'\$': '', ',': ''}, regex=True, inplace=True)
listingsgz_df_lesscol["price"] 

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
6550    120.00
6551     94.00
6552    135.00
6553    216.00
6554     90.00
Name: price, Length: 6555, dtype: object

In [23]:
listings_df.isnull().sum()[listings_df.isnull().sum() > 0].sort_values(ascending=False)

neighbourhood_group    6555
license                6535
price                  1447
last_review            1314
reviews_per_month      1314
host_name                 1
dtype: int64

In [26]:
listings_df_lesscol = listings_df.drop(["license", "neighbourhood_group", "host_name"], axis=1)

In [27]:
listings_df_lesscol.head()  

,id,name,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,97945,Rental unit in Munich · ★4.84 · 1 bedroom · 1 ...,517685,Hadern,48.11492,11.48954,Entire home/apt,80.0,2,127,2019-10-03,0.85,2,72,0
1,114695,Rental unit in Munich · ★4.82 · 1 bedroom · 1 ...,581737,Berg am Laim,48.12071,11.63758,Entire home/apt,105.0,2,66,2023-10-25,0.43,3,166,11
2,127383,Rental unit in Munich · ★4.83 · 1 bedroom · 1 ...,630556,Maxvorstadt,48.15199,11.56482,Entire home/apt,120.0,3,110,2023-12-12,0.72,1,5,3
3,159634,Rental unit in Munich · ★4.63 · 1 bedroom · 1 ...,765694,Pasing-Obermenzing,48.13855,11.46586,Entire home/apt,60.0,3,37,2023-08-09,0.25,1,5,1
4,170154,Home in Munich · ★4.96 · 1 bedroom · 2 beds · ...,108297,Sendling-Westpark,48.10814,11.52733,Private room,70.0,3,546,2023-12-19,3.66,1,169,25


In [60]:
listings_df_lesscol["price"].isnull().sum()


np.int64(1447)

In [61]:
listingsgz_df_lesscol["price"].isnull().sum() 

np.int64(1447)

#### Valores nulos price en Listings

In [62]:
listings_df_lesscol["price"].median()

np.float64(112.0)

In [65]:
median_prices_listings = listings_df_lesscol.groupby('room_type')['price'].median()
median_prices_listings

room_type
Entire home/apt    130.0
Hotel room         131.0
Private room        80.0
Shared room         70.0
Name: price, dtype: float64

In [66]:
listings_df_lesscol["price"].isnull().sum()

np.int64(1447)

In [67]:
# Calcular la mediana de cada room_type
median_prices = median_prices_listings.to_dict()

# Completar los valores nulos en la columna price con la mediana correspondiente a su room_type
listings_df_lesscol['price'] = listings_df_lesscol['price'].fillna(listings_df_lesscol['room_type'].map(median_prices))

In [68]:
listings_df_lesscol["price"].isnull().sum()

np.int64(0)

In [71]:
listings_df_lesscol[['price', 'room_type']]

,price,room_type
0,80.0,Entire home/apt
1,105.0,Entire home/apt
2,120.0,Entire home/apt
3,60.0,Entire home/apt
4,70.0,Private room
...,...,...
6550,80.0,Entire home/apt
6551,96.0,Entire home/apt
6552,76.0,Entire home/apt
6553,74.0,Entire home/apt


In [72]:
listings_df_lesscol["price"].median()

np.float64(123.0)